# 3a. Importing Libraries

In [1]:
# 3a. Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import dask.dataframe as dd

# 3b. Importing datasets

In [2]:
# 3b. Import dataset
path = r'C:\Users\navya\Instacart Basket Analysis'
df_491 = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)
df_prepared = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_4_8_done.pkl'))

## 4. Data Wrangling

In [3]:
# 4a. Look at column names and check that rename function worked
df_491.head(1)

user_id First Name    Surnam  Gender     STATE  Age date_joined  \
0    26711    Deborah  Esquivel  Female  Missouri   48    1/1/2017   

   n_dependants fam_status  income  
0             3    married  165665

In [4]:
# 4b. Rename Surnam column to Last Name
df_491.rename(columns = {'Surnam' : 'Last Name'}, inplace = True)

In [5]:
# 4c. Remove all caps in STATE column
df_491.rename(columns = {'STATE' : 'State'}, inplace = True)

In [6]:
# 4d. Clarify that n_dependants is the number of dependants
df_491.rename(columns = {'n_dependants' : 'number_dependants'}, inplace = True)

In [7]:
# 4e Drop date_joined and gender because they are not relevant to this analysis
df_491_1 = df_491.drop(columns = {'date_joined', 'Gender'})

In [8]:
# Check memory usage
df_491_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   user_id            206209 non-null  int64 
 1   First Name         194950 non-null  object
 2   Last Name          206209 non-null  object
 3   State              206209 non-null  object
 4   Age                206209 non-null  int64 
 5   number_dependants  206209 non-null  int64 
 6   fam_status         206209 non-null  object
 7   income             206209 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 12.6+ MB


In [9]:
# change all integers to 8-bit to save memory during further analysis
df_491_1['user_id'] = df_491_1['user_id'].astype('int8')
df_491_1['Age'] = df_491_1['Age'].astype('int8')
df_491_1['number_dependants'] = df_491_1['number_dependants'].astype('int8')
df_491_1['income'] = df_491_1['income'].astype('int8')
df_491_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   user_id            206209 non-null  int8  
 1   First Name         194950 non-null  object
 2   Last Name          206209 non-null  object
 3   State              206209 non-null  object
 4   Age                206209 non-null  int8  
 5   number_dependants  206209 non-null  int8  
 6   fam_status         206209 non-null  object
 7   income             206209 non-null  int8  
dtypes: int8(4), object(4)
memory usage: 7.1+ MB


## 5. Data Quality and Consistency Checks

### Convert any mixed-type data:

In [10]:
# Identify mixed-type columns
for col in df_491_1.columns.tolist():
  weird = (df_491_1[[col]].applymap(type) != df_491_1[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_491_1[weird]) > 0:
    print (col)

First Name


In [11]:
# Assign the First Name column to string type
df_491_1['First Name'] = df_491_1['First Name'].astype('str')

In [12]:
# Check the First Name column was correctly assigned to object data type
df_491_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   user_id            206209 non-null  int8  
 1   First Name         206209 non-null  object
 2   Last Name          206209 non-null  object
 3   State              206209 non-null  object
 4   Age                206209 non-null  int8  
 5   number_dependants  206209 non-null  int8  
 6   fam_status         206209 non-null  object
 7   income             206209 non-null  int8  
dtypes: int8(4), object(4)
memory usage: 7.1+ MB


### Check for missing values:

In [13]:
# Look for missing values
df_491_1.isnull().sum()

user_id              0
First Name           0
Last Name            0
State                0
Age                  0
number_dependants    0
fam_status           0
income               0
dtype: int64

#### There are no missing values in the customers dataset

### Check for duplicate values:

In [14]:
df_491_1_dups = df_491_1[df_491.duplicated()]

In [15]:
df_491_1_dups

Empty DataFrame
Columns: [user_id, First Name, Last Name, State, Age, number_dependants, fam_status, income]
Index: []

#### There are no duplicate values in the customers dataset. If there had been, they could have been dropped using the .drop() function.

In [16]:
df_491_1.head(1)

user_id First Name Last Name     State  Age  number_dependants fam_status  \
0       87    Deborah  Esquivel  Missouri   48                  3    married   

   income  
0      33

In [17]:
df_prepared.head(1)

order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0        65        1    prior             1          2                  8   

   days_since_prior_order  product_id  add_to_cart_order  reordered  True  \
0                     8.0           4                  2          0  both   

                                        product_name  aisle_id  department_id  \
0  Smart Ones Classic Favorites Mini Rigatoni Wit...        38              1   

   prices _merge  max_order  loyalty_flag spender_status order_frequency  
0    10.5   both         99  New customer   High spender             7.0

In [18]:
df_prepared.rename(columns = {'order_dow' : 'order_day_of_week'}, inplace = True)

In [19]:
#df_prepared.info()

In [20]:
df_prepared['department_id'] = df_prepared['department_id'].astype('int8')
df_prepared['prices'] = df_prepared['prices'].astype('float16')
df_prepared['loyalty_flag'] = df_prepared['loyalty_flag'].astype('category')
df_prepared['spender_status'] = df_prepared['spender_status'].astype('category')
df_prepared['order_frequency'] = df_prepared['order_frequency'].astype('category')
df_prepared['product_name'] = df_prepared['product_name'].astype('category')
df_prepared['days_since_prior_order'] = df_prepared['days_since_prior_order'].astype('int8')
df_prepared_1 = df_prepared.drop(columns = 'True')
df_prepared_2 = df_prepared_1.drop(columns = 'eval_set')
df_prepared_3 = df_prepared_2.drop(columns = '_merge')
df_prepared_4 = df_prepared_3.drop(columns = 'order_id')
df_prepared_5 = df_prepared_4.drop(columns = 'add_to_cart_order')
df_prepared_6 = df_prepared_5.drop(columns = 'reordered')
df_prepared_7 = df_prepared_6.drop(columns = 'product_id')
df_prepared_8 = df_prepared_7.drop(columns = 'aisle_id')
df_prepared_9 = df_prepared_8.drop(columns = 'product_name')
df_prepared_9.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16471603 entries, 0 to 16471602
Data columns (total 11 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   user_id                 int8    
 1   order_number            int8    
 2   order_day_of_week       int8    
 3   order_hour_of_day       int8    
 4   days_since_prior_order  int8    
 5   department_id           int8    
 6   prices                  float16 
 7   max_order               int8    
 8   loyalty_flag            category
 9   spender_status          category
 10  order_frequency         category
dtypes: category(3), float16(1), int8(7)
memory usage: 188.5 MB


In [21]:
df_491_1['fam_status'] = df_491_1['fam_status'].astype('category')
df_491_1['State'] = df_491_1['State'].astype('category')
df_491_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   user_id            206209 non-null  int8    
 1   First Name         206209 non-null  object  
 2   Last Name          206209 non-null  object  
 3   State              206209 non-null  category
 4   Age                206209 non-null  int8    
 5   number_dependants  206209 non-null  int8    
 6   fam_status         206209 non-null  category
 7   income             206209 non-null  int8    
dtypes: category(2), int8(4), object(2)
memory usage: 4.3+ MB


In [22]:
df_491_1.dtypes

user_id                  int8
First Name             object
Last Name              object
State                category
Age                      int8
number_dependants        int8
fam_status           category
income                   int8
dtype: object

In [23]:
df_prepared_9.dtypes

user_id                       int8
order_number                  int8
order_day_of_week             int8
order_hour_of_day             int8
days_since_prior_order        int8
department_id                 int8
prices                     float16
max_order                     int8
loyalty_flag              category
spender_status            category
order_frequency           category
dtype: object

In [27]:
# had to use sample to avoid memory error
df1 = df_491_1[:100000]

In [28]:
# had to use sample to avoid memory error
df2 = df_prepared_9[:100000]

## 6. Merge Data

In [29]:
# merge dataframes to include customer data in orders and products data
df_merged = df1.merge(df2, on = 'user_id')

In [30]:
df_merged.tail()

user_id First Name Last Name           State  Age  \
39056929       57    Jessica       Fox  North Carolina   31   
39056930       57    Jessica       Fox  North Carolina   31   
39056931       57    Jessica       Fox  North Carolina   31   
39056932       57    Jessica       Fox  North Carolina   31   
39056933       57    Jessica       Fox  North Carolina   31   

          number_dependants fam_status  income  order_number  \
39056929                  0     single      39            10   
39056930                  0     single      39             7   
39056931                  0     single      39             9   
39056932                  0     single      39            23   
39056933                  0     single      39            24   

          order_day_of_week  order_hour_of_day  days_since_prior_order  \
39056929                  3                 18                      10   
39056930                  5                 14                      19   
39056931                  4                 11                       5   
39056932                  4                  9                      14   
39056933                  3                 13                       6   

          department_id  prices  max_order      loyalty_flag spender_status  \
39056929              1    10.5         99      New customer   High spender   
39056930              1    10.5         99      New customer   High spender   
39056931              1    10.5         99      New customer   High spender   
39056932              1    10.5         99  Regular customer   High spender   
39056933              1    10.5         99  Regular customer   High spender   

            order_frequency  
39056929  Frequent customer  
39056930   Regular customer  
39056931  Frequent customer  
39056932   Regular customer  
39056933  Frequent customer

## 7. Export Data

In [32]:
# export data to pickle file
df_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'df_4_9_1.pkl'))